In [1]:
import matplotlib
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras import callbacks
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
from PIL import Image
from osgeo import gdal
import livelossplot

D:\Anaconda\envs\tcc\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = "clippedcrop"
modelName = "cropsNet.model"
nameLabel = "CropsLabels.pickle"

In [3]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 8
IMAGE_DIMS = (7, 55, 55)

In [4]:
data = []
labels = []

In [5]:
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)

In [6]:
count = 0
for imagePath in imagePaths:
    print(imagePath)
    raster = gdal.Open(imagePath)
    arr = raster.ReadAsArray()
    [bands,cols,rows] = arr.shape
    count += 1
    data.append(arr)
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

clippedcrop\orange\orange19.tif
clippedcrop\sugarcane\sugarcane13.tif
clippedcrop\sugarcane\sugarcane22.tif
clippedcrop\soybean\soybean35.tif
clippedcrop\soybean\soybean10.tif
clippedcrop\orange\orange25.tif
clippedcrop\soybean\soybean22.tif
clippedcrop\sugarcane\sugarcane23.tif
clippedcrop\sugarcane\sugarcane24.tif
clippedcrop\soybean\soybean40.tif
clippedcrop\orange\orange9.tif
clippedcrop\orange\orange3.tif
clippedcrop\orange\orange38.tif
clippedcrop\sugarcane\sugarcane37.tif
clippedcrop\orange\orange4.tif
clippedcrop\soybean\soybean11.tif
clippedcrop\soybean\soybean23.tif
clippedcrop\sugarcane\sugarcane5.tif
clippedcrop\sugarcane\sugarcane17.tif
clippedcrop\orange\orange27.tif
clippedcrop\soybean\soybean33.tif
clippedcrop\orange\orange12.tif
clippedcrop\soybean\soybean26.tif
clippedcrop\soybean\soybean39.tif
clippedcrop\orange\orange43.tif
clippedcrop\soybean\soybean7.tif
clippedcrop\orange\orange17.tif
clippedcrop\orange\orange11.tif
clippedcrop\soybean\soybean9.tif
clippedcrop\so

In [7]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

lb = LabelBinarizer()
labels = lb.fit_transform(labels)


[INFO] data matrix: 21.18MB


In [8]:
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.3, random_state=42)


aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="reflect")

In [9]:
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
compile1 = model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=['accuracy'])

Passou aqui1
Passou aqui2
Passou aqui3
Passou aqui4
Passou aqui5
Passou aqui6


In [10]:
#model.save(modelName)

In [11]:
f = open(nameLabel, "wb")
f.write(pickle.dumps(lb))
f.close()

In [12]:
check_p = callbacks.ModelCheckpoint(filepath='cropclassifier_cnn_{val_acc:.2f}.model',
                                    monitor='val_acc', verbose=1,
                                    save_best_only=True, save_weights_only=False)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3,
                                        verbose=1, cooldown=2)
callb_l = [check_p, reduce_lr]

plot_losses = livelossplot.PlotLossesKeras()

In [13]:
model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
    callbacks = callb_l,
	epochs=EPOCHS, verbose=0)

D:\Anaconda\envs\tcc\lib\site-packages\keras_preprocessing\image.py:1437: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (89, 7, 55, 55) (55 channels).
  str(self.x.shape[channels_axis]) + ' channels).')



Epoch 00001: val_acc improved from -inf to 0.35897, saving model to cropclassifier_cnn_0.36.model

Epoch 00002: val_acc did not improve from 0.35897

Epoch 00003: val_acc improved from 0.35897 to 0.48718, saving model to cropclassifier_cnn_0.49.model

Epoch 00004: val_acc improved from 0.48718 to 0.48718, saving model to cropclassifier_cnn_0.49.model

Epoch 00005: val_acc improved from 0.48718 to 0.51282, saving model to cropclassifier_cnn_0.51.model

Epoch 00006: val_acc improved from 0.51282 to 0.82051, saving model to cropclassifier_cnn_0.82.model

Epoch 00007: val_acc did not improve from 0.82051

Epoch 00008: val_acc improved from 0.82051 to 0.94872, saving model to cropclassifier_cnn_0.95.model

Epoch 00009: val_acc did not improve from 0.94872

Epoch 00010: val_acc did not improve from 0.94872

Epoch 00011: val_acc did not improve from 0.94872

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.

Epoch 00012: val_acc did not improve from 0.94872

Epo

In [14]:
model.save(filepath=r'cropclassifier_cnn.h5', overwrite=True)

In [15]:
model.save(modelName)